# 5.3 langchain-mcp-adpater를 활용한 MCP 도구 사용

- [MCP Transport](https://modelcontextprotocol.io/specification/2025-06-18/basic/transports)에서 SSE가 deprecate되고 Stremable HTTP가 표준으로 자리잡음
- [GitHub](https://github.com/github/github-mcp-server)에서 제공하는 MCP 서버를 Streamable HTTP와 STDIO 방식으로 활용하는 방법

- 환경변수 로딩

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

## GitHub MCP Client 설정

GitHub MCP 서버에 연결하기 위한 클라이언트 설정

1. **Streamable HTTP**: SSE를 대체하는 새로운 표준 transport 방식
2. **STDIO**: Docker를 통해 실행되는 전통적인 방식


### 설정 상세 설명

**환경 변수 요구사항:**
- `GITHUB_PAT`: GitHub Personal Access Token이 필요 (`.env` 파일에 설정)

**Docker 설정 포인트:**
- `context,pull_requests` toolsets을 활성화하여 GitHub 컨텍스트 정보와 PR 관련 작업 수행
- 컨테이너는 실행 후 자동으로 제거 (`--rm` 플래그)


In [ ]:
import os  # 환경 변수 접근을 위한 os 모듈 임포트
from langchain_mcp_adapters.client import MultiServerMCPClient  # MCP 클라이언트 어댑터 임포트

# 환경 변수에서 GitHub Personal Access Token 가져오기
github_pat = os.getenv("GITHUB_PAT")

# GitHub 서버 구성으로 MCP 클라이언트 생성
mcp_client = MultiServerMCPClient({
    # Streamable HTTP transport 옵션
    # SSE를 대체하는 새로운 transport 방식
    # "github": {
    #   "url": "https://api.githubcopilot.com/mcp/",  # GitHub Copilot MCP API 엔드포인트
    #   "headers": {
    #     "Authorization": f"Bearer {github_pat}"  # Bearer 토큰 인증
    #   },
    #   "transport": "streamable_http"  # 새로운 표준 transport 방식
    # }
   
   # Docker를 사용한 STDIO transport 
   "github": {
      "command": "docker",  # MCP 서버 실행을 위한 Docker 사용
      "args": [  # Docker 명령어 인자들
        "run",  # 새 컨테이너 실행
        "-i",   # STDIN 열어두기 (인터랙티브 모드)
        "--rm", # 컨테이너 종료 후 자동 제거
        "-e",   # 환경 변수 설정 플래그
        "GITHUB_TOOLSETS",  # 환경 변수 이름
        "-e",   # 다른 환경 변수 설정 플래그
        "GITHUB_PERSONAL_ACCESS_TOKEN",  # 환경 변수 이름
        "ghcr.io/github/github-mcp-server"  # GitHub Container Registry의 Docker 이미지
      ],
      "env": {  # 컨테이너에 전달할 환경 변수들
        "GITHUB_TOOLSETS": "context,pull_requests",  # context와 PR toolsets 활성화
        "GITHUB_PERSONAL_ACCESS_TOKEN": github_pat  # GitHub PAT를 컨테이너에 전달
      },
      "transport": "stdio"  # 통신을 위한 STDIO transport 사용
    }
})

In [17]:
tool_list = await mcp_client.get_tools()

In [19]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment='gpt-4o-2024-11-20',
    api_version='2024-08-01-preview',   
    temperature=0
)

In [20]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=tool_list,
    state_modifier=("Use the tools provided to you to answer the user's question")
)

In [ ]:
async def process_stream(stream_generator):
    results = []
    try:
        async for chunk in stream_generator:

            key = list(chunk.keys())[0]
            
            if key == 'agent':
                # Agent 메시지의 내용을 가져옴. 메세지가 비어있는 경우 어떤 도구를 어떻게 호출할지 정보를 가져옴
                content = chunk['agent']['messages'][0].content if chunk['agent']['messages'][0].content != '' else chunk['agent']['messages'][0].additional_kwargs
                print(f"'agent': '{content}'")
            
            elif key == 'tools':
                # 도구 메시지의 내용을 가져옴
                for tool_msg in chunk['tools']['messages']:
                    print(f"'tools': '{tool_msg.content}'")
            
            results.append(chunk)
        return results
    except Exception as e:
        print(f"Error processing stream: {e}")
        return results

In [23]:
from langchain_core.messages import HumanMessage

query = """깃헙의 Pull Request를 확인하고 코드 리뷰를 작성해주세요. 
코드리뷰를 PR에 comment로 남겨주세요 

PR URL:https://github.com/jasonkang14/sat-reading-client/pull/4

"""

stream_generator = agent.astream({'messages': [HumanMessage(content=query)]})


all_chunks = await process_stream(stream_generator)


if all_chunks:
    final_result = all_chunks[-1]
    print(final_result)

'agent': '{'tool_calls': [{'id': 'call_UDZyQwas02TuFPlsgrDQVlEI', 'function': {'arguments': '{"owner":"jasonkang14","repo":"sat-reading-client","pullNumber":4}', 'name': 'get_pull_request'}, 'type': 'function'}], 'refusal': None}'
'tools': '{"id":2660757826,"number":4,"state":"open","locked":false,"title":"extend splash display to 5 seconds","created_at":"2025-07-12T02:38:36Z","updated_at":"2025-07-12T02:39:08Z","user":{"login":"jasonkang14","id":45306565,"node_id":"MDQ6VXNlcjQ1MzA2NTY1","avatar_url":"https://avatars.githubusercontent.com/u/45306565?v=4","html_url":"https://github.com/jasonkang14","gravatar_id":"","type":"User","site_admin":false,"url":"https://api.github.com/users/jasonkang14","events_url":"https://api.github.com/users/jasonkang14/events{/privacy}","following_url":"https://api.github.com/users/jasonkang14/following{/other_user}","followers_url":"https://api.github.com/users/jasonkang14/followers","gists_url":"https://api.github.com/users/jasonkang14/gists{/gist_id}","